In [2]:
### Uncomment to install these packages if need be
# !pip install fooddatacentral
# !pip install pint

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets
import requests
from urllib.parse import quote
import regex as re
import fooddatacentral as fdc
from  scipy.optimize import linprog as lp
import warnings

# Supresses Warnings - Makes the Code clean in the Master Notebook
warnings.filterwarnings("ignore")

### KEEP COMMENTED, WILL ERROR OUT IF UNCOMMENTED 
import config # Imports a private API Key to test the functions

# Team Casimir Funk Nutritional Content Notebook
In this notebook, we create our nutritional content functions. Moreover, we add the nutritional data to our food prices dataset.

## Nutritional Content Functions

In [7]:
def handle_query_nc(query, food_df_dict, api_key, num_results = 10):
    '''
    Description
    --------------------------------------------------
    This is a function to handle the Food Query
    for our user.
    
    Inputs
    --------------------------------------------------
    + query : string; keywords to search the USDA 
            FoodData Central API for the user's
            desired food product
    + food_df_dict : dictionary; allows for food_df
            to be implemented and updated in 
            different functions
    + api_key : string; user's FoodData Central API
            key
    + num_results : integer; the number of results 
            the user wants in their query
    
    Outputs
    --------------------------------------------------
    + food_df is displayed
    '''
    encoded_query = quote(query)
    page_number = 1
    page_size = num_results

    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={api_key}&query={encoded_query}&pageSize={page_size}&pageNumber={page_number}'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code}")
    
    food_df = pd.DataFrame(data['foods'])
    
    food_df_dict['food_df'] = food_df
    display(food_df.iloc[:, :10])

In [8]:
def get_nutritional_content(food_df, idx = 0, computation = False):
    '''
    Description
    --------------------------------------------------
    This is a function to let the user see the details
    (detail_df) and nutritional contents 
    (nutritional_df) of individual food products.
    
    Inputs
    --------------------------------------------------
    + food_df : pandas dataframe; contains all the
            details & nutritional information for the
            different food products
    + idx : integer; corresponds to the row index of
            food product the user want to look at
    + computation : boolean; determines whether the
            detail_df & nutritional_df should be 
            displayed (important for the widget), or
            if the nutritional_df should be returned
            (for the projects data analysis section)
    
    Outputs
    --------------------------------------------------
    compuation = False : 
    + detail_df & nutritional_df are displayed
    compuation = True : 
    + nutritional_df : pandas dataframe; nutritional
            content of a single food product of 
            interest
    '''
    desired_cols = ['fdcId', 'description', 'brandOwner', 'brandName', 'marketCountry']
    avail_cols = [col for col in desired_cols if col in food_df.columns]
    
    detail_df = pd.DataFrame(food_df.loc[idx, avail_cols])
    detail_df.rename(columns = {idx : 'Details'}, inplace = True)
    
    
    nutritional_df = pd.DataFrame(food_df.loc[idx, 'foodNutrients'])
    
    desired_cols2 = ['nutrientName', 'value', 'unitName', 'percentDailyValue']
    avail_cols2 = [col for col in desired_cols2 if col in nutritional_df.columns]
    nutritional_df = nutritional_df[avail_cols2]
    
    if computation:
        return nutritional_df
    else:
        display(detail_df)
        display(nutritional_df)

In [9]:
def interactive_query(api_key):
    '''
    Description
    --------------------------------------------------
    This is a function that makes handle_query_nc()
    interactive.

    Inputs
    --------------------------------------------------
    + api_key : string; user's FoodData Central API
            key
    
    Outputs
    --------------------------------------------------
    + food_df_dict : dictionary; allows for food_df
            to be implemented and updated in 
            different functions
    '''
    food_df_dict = {}
    widget = interactive(handle_query_nc, 
                               food_df_dict = widgets.fixed(food_df_dict),
                               query = widgets.Text(description='Enter Query Here'),  
                               num_results = (1, 20, 1),
                               api_key = widgets.fixed(api_key)
                        )
    display(widget)
    return food_df_dict

In [10]:
def interactive_get_nc(food_df_dict):
    '''
    Description
    --------------------------------------------------
    This is a function that makes 
    get_nutritional_content() interactive.

    Inputs
    --------------------------------------------------
    + food_df_dict : dictionary; allows for food_df
            to be implemented and updated in 
            different functions
    
    Outputs
    --------------------------------------------------
    + Interactive widget is displayed
    '''
    widget = interactive(get_nutritional_content, food_df = widgets.fixed(food_df_dict['food_df']), 
                  idx = widgets.IntSlider(value = 0, min = 0, max = len(food_df_dict['food_df']) - 1, step = 1, description='Food Index'), 
                         computation = widgets.fixed(False)
                        )
    display(widget)

## Joining Our Food Price and NC Data

The first step is to load in the Food Price Data that we manually collected before we automate the process of adding the nutritional data for all of the food products at the stores within our analysis scope.

In [13]:
price = pd.read_csv('./data/food_prices.csv', dtype=str).set_index('Food item')
price.iloc[:5, :11]

,GTIN/UPC of TJs,Price at TJs ($),Quantity,Unit,Brand/Type at TJs,GTIN/UPC of Safeway,Price at Safeway ($),Quantity,Unit.1,Brand/Type at Safeway,GTIN/UPC of Berkeley Bowl
Food item,,,,,,,,,,,
Bread,00740661,4.49,24.00,oz,Sprouted Wheat Sourdough Bread,021130183432,4.99,24,oz,Signature SELECT Bread Long Sliced Loaf San Fr...,073130002602
Rice,017400118006,3.29,3.00,lb,TJs Jasmine Rice,21130502097,6.79,5,lb,Mahatma Jasmine Thai Fragrant Long Grain Rice ...,688267090882
Pasta,00323673,1.99,1.00,lb,Organic Fusilli Corti Bucati Pasta,021130507368,1.50,16,oz,Signature SELECT Pasta Penne Rigate Box,076808006575
Apples,887434088381,1.29,0.33,lb,Honeycrisp Apples,887434088381,2.00,0.33,lb,Honeycrisp apples,887434088381
Tomatoes,021130338023,2.99,1.00,lb,TJs Campari Tomatoes,021130338023,3.49,1,lb,Red roma tomato,021130338023


Examining a subset of our data, it becomes evident that our data is not in a format in which we will be able to quickly pull all of the requisite nutritional data, we have to many columns and each row is not unique to a specific food product at a specific store. Thus, we must reformat the data such that we only have seven columns:
1. `Food Item` : The food product
2. `Store` : The store in which the `Food Item` is sold
3. `GTIN/UPC` : A unique identifier in the USDA FoodCentral database for the `Food Item` from a specific `Store`
4. `Price` : The price of the `Food Item`
5. `Quantity` : The quantity of the `Food Item`
6. `unit` : The units of the `Food Item`
7. `Brand/Type` : The name of the `Food Item`

In order to achieve this desired format we will go through the following process:
1. Create a dictionary `price_df_dict` where we will keep individual dataframes for each of the stores in question with information on the aformentioned variables.
2. Combine all of these sub-dataframes into one large one `price_rf` that has all of our information in our desired format.

In [15]:
price_df_dict = {}

stores = price.filter(regex = '^Brand').columns
stores = [re.findall(r'Brand/Type at (.+)', col)[0] for col in stores]

num_sub_df_cols = 5
num_sub_dfs = int(price.shape[1] / num_sub_df_cols)

for i in range(len(stores)):
    store = stores[i]
    back_idx = num_sub_df_cols*(i + 1)
    sub_df = price.iloc[:, back_idx - num_sub_df_cols : back_idx]
    sub_df['Store'] = store
    sub_df.columns = ['GTIN/UPC', 'Price', 'Quantity', 'Unit', 'Brand/Type', 'Store']
    sub_df = sub_df[['Store', 'GTIN/UPC', 'Price', 'Quantity', 'Unit', 'Brand/Type']]
    sub_df.reset_index(inplace = True)
    price_df_dict[store] = sub_df

price_rf = pd.DataFrame()

for key in price_df_dict.keys():
    if len(price_rf) == 0:
        price_rf =  price_df_dict[key]
    else:
        price_rf = pd.concat([price_rf, price_df_dict[key]]).reset_index(drop = True)

In [16]:
price_rf

,Food item,Store,GTIN/UPC,Price,Quantity,Unit,Brand/Type
0,Bread,TJs,00740661,4.49,24.00,oz,Sprouted Wheat Sourdough Bread
1,Rice,TJs,017400118006,3.29,3.00,lb,TJs Jasmine Rice
2,Pasta,TJs,00323673,1.99,1.00,lb,Organic Fusilli Corti Bucati Pasta
3,Apples,TJs,887434088381,1.29,0.33,lb,Honeycrisp Apples
4,Tomatoes,TJs,021130338023,2.99,1.00,lb,TJs Campari Tomatoes
...,...,...,...,...,...,...,...
183,Lentils,Whole Foods,024182021028,4.79,16,oz,Organic Green Lentils
184,Brussels Sprouts,Whole Foods,093966907162,4.99,16,oz,Brussel Sprout Organic
185,Edamame,Whole Foods,041268172030,3.99,1,lb,Shelled Edamame
186,Tofu,Whole Foods,099482409258,4.99,10,oz,Hodo Extra Firm Organic Tofu


Now that our data is properly formated, we can now add our nutritional data to our food prices dataset.

### Pull Dietary Reference Categories
Using the `handle_query_dr` function from our `dietary_references` notebook we are able to get all of the categories for which we need the nutritional content. We obtain all these categories by finding the dietary minimums and maximums put forth by the USDA.

In [19]:
%run dietary_references.ipynb

In [269]:
### Note the DR categories are the same for ALL Age-Sex Groups.
### It is the values that are different
dmins = handle_query_dr(1, 'Male')[1][['Nutrition']]
dmaxs = handle_query_dr(1, 'Male', 'max')[1][['Nutrition']]
drs = pd.concat([dmins, dmaxs]).reset_index(drop = True)

### We need to add 'Vitamin A, IU' as many 
### of the foods with Vitamin A have it in IU units
drs.loc[len(drs)] = {'Nutrition' : 'Vitamin A, IU'}

### Get Nutritional Contents

When originally going about our research, we utilized the functions below to query the USDA FoodData Central API and pull the requisite nutritional contents for our food products. However, doing so takes a considerable amount of time. Therefore, in order to speed up and faciliate the usage of this notebook, especially when using the widgets in the master notebook, we have provided and read in a `.csv` file called `price_master.csv` that contains all of the information we compiled from the API. If you wish, feel free to run the code that we used to create `price_master.csv` by uncommenting the code in the cell that reads in file. <br>
**Warning:** The code may take up to a couple of minutes to run, so re-comment the code after running it in order to avoid unintentionally re-running it.

In [261]:
def handle_query_nc_calc(query, api_key):
    '''
    Description
    --------------------------------------------------
    This is a function is an adaptation of our
    handle_query_nc() function that is optimized for the
    task of pulling nutritional data
    
    Inputs
    --------------------------------------------------
    + query : string; keywords to search the USDA 
            FoodData Central API for the user's
            desired food product
    + api_key : string; user's FoodData Central API
            key
    
    Outputs
    --------------------------------------------------
    + food_df : pandas dataframe; contains all the
            details & nutritional information for the
            different food products
    '''
    query = str(query)
    encoded_query = quote(query)
    page_number = 1
    page_size = 1

    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={api_key}&query={encoded_query}&pageSize={page_size}&pageNumber={page_number}'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code}")
    
    food_df = pd.DataFrame(data['foods'])
    
    return food_df

In [263]:
def compile_ncs(ncs_master, fp_arr, idx, API_KEY, default_df = price_rf):
    '''
    Description
    --------------------------------------------------
    This is a function to let the user see the details
    (detail_df) and nutritional contents 
    (nutritional_df) of individual food products.
    
    Inputs
    --------------------------------------------------
    + ncs_master : pandas dataframe; contains all the
            details & nutritional information for the
            different food products in our food prices
            dataset
    + idx : integer; corresponds to the index of
            food product we want to pull the 
            nutritional content for
    + API_KEY : string; user's FoodData Central API
            key
    + defaul_df : pandas dataframe; in the case of a
            bad query (e.g. inncorrect GTIN), the 
            default food_item can be pulled and 
            have query performed on it
    Outputs
    --------------------------------------------------
    + ncs_master : pandas dataframe; udpadted version
            of ncs_master with the nutritional 
            information of the food product at index 
            idx
    '''  
    try: # Catches bad queries
        fp = fp_arr[idx]
        food_df = handle_query_nc_calc(fp, API_KEY)
        ncs = get_nutritional_content(food_df, idx = 0, computation = True)
        ncs = ncs.drop_duplicates(subset='nutrientName', keep='first')
        ncs = ncs.reset_index(drop = True)
    except: # Uses default food item
        fp = default_df.loc[idx, 'Food item']
        food_df = handle_query_nc_calc(fp, API_KEY)
        ncs = get_nutritional_content(food_df, idx = 0, computation = True)
        ncs = ncs.drop_duplicates(subset='nutrientName', keep='first')
        ncs = ncs.reset_index(drop = True)
    
    ncs_m = drs.merge(ncs, how = 'left', left_on = 'Nutrition', right_on = 'nutrientName')[['Nutrition', 'value']]
    ncs_m.columns = ['Nutrient', 'Value']
    ncs_m['Value'] = ncs_m['Value'].fillna(0)

    ncs_t = ncs_m.transpose()
    ncs_t.columns = ncs_t.iloc[0, :]
    ncs_t = ncs_t.iloc[1:, :]
    ncs_rf = pd.DataFrame(ncs_t.loc['Value', :]).transpose()
    ncs_rf = ncs_rf.reset_index(drop = True)
    ncs_rf.columns.name = None
         
    
    if len(ncs_master) > 0:
        ncs_master = pd.concat([ncs_master, ncs_rf], ignore_index = True)
    else:
        ncs_master = ncs_rf
    
    return ncs_master

In [293]:
price_master = pd.read_csv('./data/price_master.csv', dtype=str)

### Only uncomment if using, otherwise, DO NOT UNCOMMENT! 
### Code takes a lot of time to run and do not want to 
### exceed API call rate limits
# ncs_master = pd.DataFrame()
# search_col = 'GTIN/UPC'
# API_KEY = config.API_KEY # Enter your own API Key
# fp_arr = price_rf[search_col]

# for i in range(len(fp_arr)):
#     ncs_master = compile_ncs(ncs_master, fp_arr, i, API_KEY)

# price_master = pd.concat([price_rf, ncs_master], axis = 1)
# price_master = price_master.dropna().reset_index(drop = True)
# price_master['Vitamin A, RAE'] = price_master['Vitamin A, RAE'] + price_master['Vitamin A, IU']*0.3
# price_master = price_master.drop(columns = ['Vitamin A, IU'])

In [295]:
price_master.to_csv('./data/price_master.csv', index = False)

In [297]:
# Ensures the nutritional data values are floats & not strings
price_master.iloc[:, 7:] = price_master.iloc[:, 7:].astype(float)
price_master['Price'] = price_master['Price'].astype(float)
price_master['FDC Quantity'] = price_master[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
price_master['FDC Price'] = price_master['Price'] / price_master['FDC Quantity']
price_master.head()

,Food item,Store,GTIN/UPC,Price,Quantity,Unit,Brand/Type,Energy,Protein,"Fiber, total dietary",...,Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn","Sodium, Na",Energy.1,FDC Quantity,FDC Price
0,Bread,TJs,00740661,4.49,24.00,oz,Sprouted Wheat Sourdough Bread,214.0,17.9,10.7,...,0.0,0.0,0.0,0.0,0.0,0.0,464.0,214.0,6.80388555 hectogram,0.6599170381400669 / hectogram
1,Rice,TJs,017400118006,3.29,3.00,lb,TJs Jasmine Rice,370.0,8.7,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.0,13.6077711 hectogram,0.2417736141960824 / hectogram
2,Pasta,TJs,00323673,1.99,1.00,lb,Organic Fusilli Corti Bucati Pasta,375.0,12.5,8.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,375.0,4.535923700000001 hectogram,0.4387199017479063 / hectogram
3,Apples,TJs,887434088381,1.29,0.33,lb,Honeycrisp Apples,54.0,0.41,2.1,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,54.0,1.4968548210000003 hectogram,0.8618070249045213 / hectogram
4,Tomatoes,TJs,021130338023,2.99,1.00,lb,TJs Campari Tomatoes,21.0,0.83,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,99.0,21.0,4.535923700000001 hectogram,0.659182163932784 / hectogram


In [325]:
desired_age = 20
desired_sex = 'Male'
age_col, diet_reqs = handle_query_dr(desired_age, desired_sex)
diet_maxs = handle_query_dr(desired_age, desired_sex, mode = 'max')[1]
diet_reqs = diet_reqs[['Nutrition', age_col]].set_index('Nutrition')
diet_maxs = diet_maxs[['Nutrition', age_col]].set_index('Nutrition').sort_index()
diet_master = pd.concat([diet_reqs,-diet_maxs])

desired_store = 'Whole Foods'
alg_df = price_master[price_master['Store'] == desired_store]
alg_prices = alg_df['Price']
alg_df = alg_df.set_index('Food item').iloc[:, 6:-2].transpose()
ad_min = alg_df.loc[diet_reqs.index]
ad_max = alg_df.loc[diet_maxs.index]
alg_df = pd.concat([ad_min,-ad_max])
alg_df.columns.name = None
alg_df.index.name = None

alg_df

,Bread,Rice,Pasta,Apples,Tomatoes,Lettuce,Eggs,Milk,Frozen Shrimp,Chicken,...,Lemons,Sweet Potatoes,Chickpeas,Kidney Beans,Parnship,Mushrooms,Lentils,Brussels Sprouts,Edamame,Tofu
Energy,241.0,370.0,375.0,54.0,21.0,18.0,140.0,47.0,107.0,134.0,...,31.0,82.0,120.0,337.0,75.0,24.0,271.0,41.0,118.0,83.0
Protein,6.9,8.7,10.7,0.41,0.83,1.18,12.0,3.39,20.5,17.9,...,0.88,1.18,7.0,21.7,1.2,3.53,22.9,3.53,10.6,9.52
"Fiber, total dietary",3.4,0.0,3.6,2.1,0.8,2.4,0.0,0.0,0.0,0.0,...,2.7,3.5,4.0,15.2,4.9,1.2,25.0,3.5,4.7,1.2
"Folate, DFE",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,67.0,0.0,0.0,0.0,0.0,0.0
"Calcium, Ca",207.0,0.0,0.0,8.0,0.0,33.0,60.0,138.0,54.0,0.0,...,26.0,35.0,23.0,87.0,36.0,0.0,42.0,42.0,71.0,131.0
"Carbohydrate, by difference",48.3,82.6,76.8,14.0,4.13,3.53,0.0,5.08,0.89,0.0,...,9.73,20.0,20.0,60.9,18.0,3.53,62.5,9.41,9.41,2.38
"Iron, Fe",2.48,0.0,3.21,0.15,0.0,0.94,2.0,0.0,1.61,0.96,...,0.0,0.59,1.0,6.65,0.59,0.42,7.71,1.18,2.12,1.19
"Magnesium, Mg",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29.0,0.0,125.0,0.0,0.0,0.0
Niacin,0.0,0.0,5.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0
"Phosphorus, P",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0


In [327]:
dummy = pd.DataFrame(alg_df.sum(axis=1))
dummy['Reqs'] = diet_master
dummy

,0,Reqs
Energy,9673.0,2400.0
Protein,426.37,56.0
"Fiber, total dietary",163.5,33.6
"Folate, DFE",67.0,400.0
"Calcium, Ca",1790.0,1000.0
"Carbohydrate, by difference",1044.76,130.0
"Iron, Fe",82.39,8.0
"Magnesium, Mg",154.0,400.0
Niacin,11.39,16.0
"Phosphorus, P",71.0,700.0


In [329]:
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) tr
result = lp(alg_prices, alg_df, -diet_master, method='highs')
result

       message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)
       success: False
        status: 2
           fun: None
             x: None
           nit: 0
         lower:  residual: None
                marginals: None
         upper:  residual: None
                marginals: None
         eqlin:  residual: None
                marginals: None
       ineqlin:  residual: None
                marginals: None

In [331]:
# print(f"Cost of diet for {age_col} is ${result.fun:.2f} per day.")